In [ ]:
from IPython.display import DisplayObject
import base64

class ES6Module(DisplayObject):
    _MIME_TYPE = 'application/vnd.jupyter.es6-rich-output'
    
    def __init__(self, module=None, url=None):
        if module and url:
            raise ValueError('Cannot specify both module and url')
        if not module and not url:
            raise ValueError('Must specify either module or url')
        if module:
            url = 'data:text/javascript;base64,' + base64.b64encode(module.encode('utf-8')).decode('utf-8') 
        self._url = url
        self.data = {}
    
    def _repr_mimebundle_(self, include=None, exclude=None):
        mime_bundle = {} 
        mime_bundle[ES6Module._MIME_TYPE] = self._url
        for key in self.data.keys():
            mime_bundle[key] = self.data[key]
        return mime_bundle

## `render` function can write to element

In [ ]:
ES6Module(module='''
export function render(output, element) {
    const div = document.createElement('div');
    div.style.background = 'lime';
    div.textContent = 'passed';
    element.appendChild(div);
}
''')

## Render function finishes before rendering next output (failing)

In [ ]:
from IPython.display import Javascript
display(Javascript('window.nextOutputRendered = false; console.log("Set rendered output false")'))
display(ES6Module(module='''
export async function render(output, element) {
    const promise = new Promise((resolve) => setTimeout(resolve, 1000));
    await promise;
    console.log("HERE123")
    if (window.nextOutputRendered) {
        const div = document.createElement('div');
        div.style.background = 'red';
        div.textContent = 'failed: expected subsequent outputs to wait for this.';
        element.appendChild(div);
    } else {
        const div = document.createElement('div');
        div.style.background = 'lime';
        div.textContent = 'passed';
        element.appendChild(div);
    }
}
'''))
display(Javascript('window.nextOutputRendered = true; console.log("Set rendered output true")'))

## Render function has access to all output data

In [ ]:
data = ES6Module(module='''
export function render(output, element) {
    const plainText = output.data['text/plain'];
    if (plainText != 'the text value') {
        const div = document.createElement('div');
        div.style.background = 'red';
        div.textContent = `Expected the text to be "the text value" but was ${plainText}`;
        element.appendChild(div);
    } else {
        const div = document.createElement('div');
        div.style.background = 'lime';
        div.textContent = 'passed';
        element.appendChild(div);
    }
}
''')
data.data['text/plain'] = 'the text value'
data

# Validate that the members of the context do not include anything extra

In [ ]:
ES6Module(module='''
const expected = {
    comms: {
        open: () => {},
        registerTarget: () => {},
    },
};

export function render(output, element, context) {
    
    let errorsReported = false;
    validate(context, expected, 'context.', (error) => {
        errorsReported = true;
        const div = document.createElement('div');
        div.style.background = 'red';
        div.textContent = error;
        element.appendChild(div);
    });
    if (!errorsReported) {
        const div = document.createElement('div');
        div.style.background = 'lime';
        div.textContent = 'passed';
        element.appendChild(div);
    }
}

function getMembers(object) {
    const members = new Set([...Object.getOwnPropertyNames(object)]);
    const prototype = Object.getPrototypeOf(object);
    if (prototype && prototype != Object.getPrototypeOf({})) {
        return new Set([...members, ...getMembers(prototype)]);
    }
    return members;
}

function validate(actual, expected, prefix, recordError) {
    const expectedKeys = new Set(Object.getOwnPropertyNames(expected));
    const actualKeys = getMembers(actual);
    
    for (const key of actualKeys) {
        console.log("CHECKING HERE1")
        if (!expectedKeys.has(key)) {
            recordError(`Unexpected key: ${prefix}${key}`);
            continue;
        }
        const actualValue = actual[key];
        const expectedValue = expected[key];
        if (typeof actualValue === 'function' && typeof expectedValue === 'function') {
            continue;
        }
        validate(actualValue, expectedValue, `${prefix}${key}.`, recordError);
    }
    
    for (const key of expectedKeys) {
        if (!actualKeys.has(key)) {
            recordError(`Missing key: ${prefix}${key}`);
            continue;
        }
    }
}
''')

# Comms

## Register comm in Python, and then open in JS

In [ ]:
def target_func(comm, msg):
    # Only send the response if it's the data we are expecting.
    if msg['content']['data'] == 'the data':
        comm.send({'response': 'got comm open!',}, None, msg['buffers']);
get_ipython().kernel.comm_manager.register_target('comm_target', target_func)

ES6Module(module='''
export async function render(output, element, context) {
    const buffer = new Uint8Array(10);
    for (let i = 0; i < buffer.byteLength; ++i) {
        buffer[i] = i
    }
    const channel = await context.comms.open('comm_target', 'the data', [buffer.buffer]);
    let success = false;
    for await (const message of channel.messages) {
        if (message.data.response == 'got comm open!') {
            if (!(message.buffers[0] instanceof ArrayBuffer)) {
                throw new Error('Buffer is not an ArrayBuffer');    
            }
            const responseBuffer = new Uint8Array(message.buffers[0]);
            for (let i = 0; i < buffer.length; ++i) {
               if (responseBuffer[i] != buffer[i]) {
                   throw new Error('comm buffer different at ' + i);
                   return;
               }
            }
            // Close the channel once the expected message is received. This should
            // cause the messages iterator to complete and for the for-await loop to
            // end.
            channel.close();
        }
    }
    const div = document.createElement('div');
    div.style.background = 'lime';
    div.textContent = 'passed';
    element.appendChild(div);
}
''')

## Register comm in JS, and then open in Python

In [ ]:
from ipykernel.comm import Comm

display(ES6Module(module='''
export async function render(output, element, context) {
    context.comms.registerTarget('comms_testing', (comm, data, buffers) => {
        comm.send('this is the response', {buffers: buffers});
    
        const div = document.createElement('div');
        div.style.background = 'lime';
        div.textContent = 'passed';
        element.appendChild(div);
    });
}
'''))


buffer = b'hello world'
comm = Comm(target_name='comms_testing', data={'foo': 1}, buffers=[buffer])

message = None
def handle_message(msg):
    global message
    message = msg

comm.on_msg(handle_message)

In [ ]:
assert message['content']['data'] == 'this is the response'
assert str(message['buffers'][0].tobytes()) == str(buffer)